# Duplication рекламы

Пример отчета по расчету пересечения аудитории медиа объектов в рамках проекта "Измерение аудитории и профиля рекламных кампаний в интернете". 

## Описание задачи

Посчитаем пересечение аудитории рекламных сетей Everest-network и Betweenx.

Общие параметры:
- Период: Декабрь 2023
- География: Россия 0+
- Население: 12+
- Типы пользования интернетом: ограничения нет, считаем по всем (Web Desktop, Web Mobile, App Mobile)

Статистики:
- Reach (reach)
- Reach% (reachPer)
- Average Daily Reach (adr)
- Average Daily Reach% (adrPer)

# Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API Cross Web и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd

from mediascope_api.core import net as mscore
from mediascope_api.crossweb import tasks as cwt
from mediascope_api.crossweb import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с API Cross Web
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.CrossWebTask()
cats = cwc.CrossWebCats()

## Формируем задания


### Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# Задаем период
# Период указывается в виде списка ('Начало', 'Конец'). Можно указать несколько периодов
date_filter = [('2023-12-01', '2023-12-31')]

# Задаем фильтр по типам пользования интернетом
usetype_filter = [1,2,3]

### Получим ID рекламных сетей.
Для построения отчета необходимо получить идентификаторы рекламных сетей __Everest-network__ и __Betweenx__.

Для этого воспользуемся методами поиска в справочнике. Работа со справочником подробно описана в ноутбуке [catalogs](catalogs.ipynb). 

Получим идентификатор рекламной сети **Everest-network** и рекламной сети **Betweenx**

In [ ]:
cats.get_ad_agency(agency='Everest-network')

In [ ]:
cats.get_ad_agency(agency='Betweenx')

Таким образом, необходимые идентификаторы следующие:

- **Everest-network** advertisementAgencyId = 12983
- **Betweenx** advertisementAgencyId = 59418

### Расчет пересечения аудитории рекламных сетей

In [ ]:
%%time

# Задаем название для отображения в DataFrame
project_name = 'Profile duplication'

# Задаем фильтр по географии, 
geo_filter = None

# Задаем фильтр по демографии
demo_filter = None

# Задаем фильтр по рекламе и фильтр по пересечению, в нашем случае это ID рекламных сетей
mart_filter = "advertisementAgencyId IN (12983)"
duplication_mart_filter = "advertisementAgencyId IN (59418)"

# Указываем список срезов, чтобы сформировать структуру расчета
slices = ['advertisementAgencyName', 'duplicationAdvertisementAgencyName', 'researchMonth']

# Указываем список статистик для расчета
statistics = ['reach', 'reachPer', 'adr', 'adrPer']

# Формируем задание для API Cross Web в формате JSON
task_json = mtask.build_task_media_duplication('profile-duplication', project_name, date_filter=date_filter, 
                             usetype_filter=usetype_filter, geo_filter=geo_filter,
                             demo_filter=demo_filter, mart_filter=mart_filter,
                             duplication_mart_filter=duplication_mart_filter,
                             slices=slices, statistics=statistics, scales=None)

# Отправляем задание на расчет и ждем выполнения
task = mtask.wait_task(mtask.send_profile_duplication_task(task_json))

# Получаем результат
df_result = mtask.result2table(mtask.get_result(task), project_name = project_name)
df_result

In [ ]:
writer = pd.ExcelWriter(mtask.get_excel_filename('Sample profile_duplication'))
df_info = mtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()